# HW7: Neural Transition-Based Dependency Parsing


In this exercise, you are going to build a deep learning model for Neural Networks Transition-Based Dependency Parsing. A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between “head” words and words which modify those heads. Your implementation will be a transition-based parser, which incrementally builds up a parse one step at a time.

To complete this exercise, you will need to complete the code and build a deep learning model for dependency parsing. We will evaluate the model on the subset of Penn Treebank (annotated with Universal Dependencies). 

We provide the code for data preparation and the skeleton for PartialParse class. You do not need to understand the code outside of this notebook. 

This homework and the starter codes are adopt from Stanford University class CS224n.

This homework does not require you to use Google Colab or Google Cloud as the model is quite small (but you can still use it if you want)

## 1. Transition-Based Dependency Parsing

Your implementation will be a transition-based parser, which incrementally builds
up a parse one step at a time. At every step it maintains a partial parse, which is represented as follows:
- A stack of words that are currently being processed.
- A buffer of words yet to be processed.
- A list of dependencies predicted by the parser.

Initially, the stack only contains ROOT, the dependencies lists is empty, and the buffer contains all words
of the sentence in order. At each step, the parse applies a transition to the partial parse until its buffer is
empty and the stack is size 1. The following transitions can be applied:
- SHIFT: removes the first word from the buffer and pushes it onto the stack.
- LEFT-ARC: marks the second (second most recently added) item on the stack as a dependent of the
first item and removes the second item from the stack.
- RIGHT-ARC: marks the first (most recently added) item on the stack as a dependent of the second
item and removes the first item from the stack.

Your parser will decide among transitions at each state using a neural network classifier.

### TODO 1 (Written):
Go through the sequence of transitions needed for parsing the sentence “I parsed
this sentence correctly”. The dependency tree for the sentence is shown below. At each step, give the
configuration of the stack and buffer, as well as what transition was applied this step and what new
dependency was added (if any). The first three steps are provided below as an example.

<img src="https://github.com/ekapolc/NLP_2020/blob/master/HW7/images/img1.jpg?raw=true">

Complete the following table (double click the table and fill in the rest):

| stack    |  buffer |  new dependency | transition |
| :------: |:------: | :-------------: | :--------: |
| \[ROOT\]            | \[I, parsed, this, sentence, correctly\] |          | Initial Configuration |
| \[ROOT, I\]         | \[parsed, this, sentence, correctly\]    |          | SHIFT |
| \[ROOT, I, parsed\] | \[this, sentence, correctly\]            |          | SHIFT |
| \[ROOT, parsed\]    | \[this, sentence, correctly\]            | parsed→I | LEFT-ARC |
|\[ROOT,parsed,this\]|\[sentence,correctly\]|          | SHIFT |
|\[ROOT,parsed,this,sentence\]|\[correctly\]| | SHIFT |
|\[ROOT,parsed,sentence\]| \[correctly\]|sentence→this| LEFT-ARC |
|\[ROOT,parsed\] |\[correctly\] |parsed→sentence| RIGHT-ARC |
|\[ROOT,parsed,correctly\] |\[\] || SHIFT |
|\[ROOT,parsed\] |\[\] |parsed→correctly| RIGHT-ARC |
|\[ROOT\] |\[\] |ROOT→parsed| RIGHT-ARC |

### TODO 2 (Coding):
Implement the __\_\_init\_\___ and __parse_step__ functions in the PartialParse class. Your code must past both of the following tests.

In [8]:
class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.

        Your code should initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the
                        last element of the list.
            self.buffer: The current buffer represented as a list with the first item on the
                         buffer as the first item of the list
            self.dependencies: The list of dependencies produced so far. Represented as a list of
                    tuples where each tuple is of the form (head, dependent).
                    Order for this list doesn't matter.

        The root token should be represented with the string "ROOT"

        Args:
            sentence: The sentence to be parsed as a list of words.
                      Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes. Do not use it in your code.
        self.sentence = sentence

        ### YOUR CODE HERE
        self.stack = ["ROOT"]
        self.buffer = list(self.sentence)
        self.dependencies = []
        ### END YOUR CODE

    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        Args:
            transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                        and right-arc transitions. You can assume the provided transition is a legal
                        transition.
        """
        ### YOUR CODE HERE
        if transition == 'S':
            self.stack.append(self.buffer.pop(0))
        elif transition == 'LA':
            dependent = self.stack.pop(-2)
            head = self.stack[-1]
            self.dependencies.append((head,dependent))
        elif transition == 'RA':
            dependent = self.stack.pop(-1)
            head = self.stack[-1]
            self.dependencies.append((head,dependent))

        ### END YOUR CODE

    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        Args:
            transitions: The list of transitions in the order they should be applied
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
                          as a list of tuples where each tuple is of the form (head, dependent)
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies

In [9]:
# Do not modify this code
def test_step(name, transition, stack, buf, deps,
              ex_stack, ex_buf, ex_deps):
    """Tests that a single parse step returns the expected output"""
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps

    pp.parse_step(transition)
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    assert stack == ex_stack, \
        "{:} test resulted in stack {:}, expected {:}".format(name, stack, ex_stack)
    assert buf == ex_buf, \
        "{:} test resulted in buffer {:}, expected {:}".format(name, buf, ex_buf)
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)
    print("{:} test passed!".format(name))


def test_parse_step():
    """Simple tests for the PartialParse.parse_step function
    Warning: these are not exhaustive
    """
    test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [],
              ("ROOT", "the", "cat"), ("sat",), ())
    test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], [],
              ("ROOT", "cat",), ("sat",), (("cat", "the"),))
    test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [],
              ("ROOT", "run",), (), (("run", "fast"),))


def test_parse():
    """Simple tests for the PartialParse.parse function
    Warning: these are not exhaustive
    """
    sentence = ["parse", "this", "sentence"]
    dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
    dependencies = tuple(sorted(dependencies))
    expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
    assert dependencies == expected,  \
        "parse test resulted in dependencies {:}, expected {:}".format(dependencies, expected)
    assert tuple(sentence) == ("parse", "this", "sentence"), \
        "parse test failed: the input sentence should not be modified"
    print("parse test passed!")

In [10]:
test_parse_step()
test_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


### TODO 3 (Coding):
Our network will predict which transition should be applied next to a partial parse. We could use it to parse a single sentence by applying predicted transitions until the parse is complete. However, neural networks run much more efficiently when making predictions about batches of data at a time (i.e., predicting the next transition for a many different partial parses simultaneously). We can parse sentences in minibatches with the following algorithm.

<img src="https://github.com/ekapolc/NLP_2020/blob/master/HW7/images/img2.jpg?raw=true">

Implement this algorithm in the minibatch parse function and run the following test.

In [46]:
def minibatch_parse(sentences, model, batch_size):
    """Parses a list of sentences in minibatches using a model.

    Args:
        sentences: A list of sentences to be parsed (each sentence is a list of words)
        model: The model that makes parsing decisions. It is assumed to have a function
               model.predict(partial_parses) that takes in a list of PartialParses as input and
               returns a list of transitions predicted for each parse. That is, after calling
                   transitions = model.predict(partial_parses)
               transitions[i] will be the next transition to apply to partial_parses[i].
        batch_size: The number of PartialParses to include in each minibatch
    Returns:
        dependencies: A list where each element is the dependencies list for a parsed sentence.
                      Ordering should be the same as in sentences (i.e., dependencies[i] should
                      contain the parse for sentences[i]).
    """

    ### YOUR CODE HERE
    partial_parses = [PartialParse(sent) for sent in sentences]
    unfinished_parses = list(partial_parses)
    while unfinished_parses:
        take = unfinished_parses[:batch_size]
        transitions = model.predict(take)
        for i,pp in enumerate(take):
            pp.parse_step(transitions[i])
        for pp in take:
            if len(pp.buffer) == 0 and len(pp.stack) == 1:
                unfinished_parses.pop(unfinished_parses.index(pp))
    dependencies = list([list(pp.dependencies) for pp in partial_parses])
    ### END YOUR CODE
    return dependencies

In [47]:
# Do not modify this code
class DummyModel(object):
    """Dummy model for testing the minibatch_parse function
    First shifts everything onto the stack and then does exclusively right arcs if the first word of
    the sentence is "right", "left" if otherwise.
    """
    def predict(self, partial_parses):
        return [("RA" if pp.stack[1] is "right" else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses]


def test_dependencies(name, deps, ex_deps):
    """Tests the provided dependencies match the expected dependencies"""
    deps = tuple(sorted(deps))
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)


def test_minibatch_parse():
    """Simple tests for the minibatch_parse function
    Warning: these are not exhaustive
    """
    sentences = [["right", "arcs", "only"],
                 ["right", "arcs", "only", "again"],
                 ["left", "arcs", "only"],
                 ["left", "arcs", "only", "again"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'right'), ('arcs', 'only'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[1],
                      (('ROOT', 'right'), ('arcs', 'only'), ('only', 'again'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[2],
                      (('only', 'ROOT'), ('only', 'arcs'), ('only', 'left')))
    test_dependencies("minibatch_parse", deps[3],
                      (('again', 'ROOT'), ('again', 'arcs'), ('again', 'left'), ('again', 'only')))
    print("minibatch_parse test passed!")

In [48]:

test_minibatch_parse()

minibatch_parse test passed!


## 2. Setup and Preprocessing

In [ ]:
!wget --no-check-certificate https://raw.githubusercontent.com/ekapolc/nlp_2020/master/HW7/HW7.zip
!unzip HW7.zip

In [53]:
from utils.parser_utils import minibatches, load_and_preprocess_data

Preparing data. We will use a subset of Penn Treebank and pretrained embeddings in this task

We are now going to train a neural network to predict, given the state of the stack, buffer, and dependencies, which transition should be applied next. First, the model extracts a feature vector representing the current state. We will be using the feature set presented in the original neural dependency parsing paper: A Fast and Accurate Dependency Parser using Neural Networks. 

The function extracting these features has been implemented for you in parser_utils. This feature vector consists of a list of tokens (e.g., the last word in the stack, first word in the buffer, dependent of the second-to-last word in the stack if there is one, etc.). They can be represented as a list of integers.

In [54]:
parser, embeddings, train_examples, dev_set, test_set = load_and_preprocess_data(True)

Loading data...
took 2.12 seconds
Building parser...
took 0.03 seconds
Loading pretrained embeddings...
took 2.25 seconds
Vectorizing data...
took 0.06 seconds
Preprocessing training data...
took 1.36 seconds


In [55]:
print(len(train_examples), len(dev_set), len(test_set))

48390 500 500


In [56]:
print(embeddings.shape)

(5157, 50)


Get the full batch of our subset data

In [57]:
minibatch_gen = minibatches(train_examples, len(train_examples))
x_train, y_train = minibatch_gen.__next__()

In [58]:
print(x_train.shape)
print(y_train.shape)

(48390, 36)
(48390, 3)


In [62]:
# Sample features
print(x_train[0])
print([parser.id2tok[i] for i in x_train[0]])

[5156 4003   86   94  822   86 5155 5155 5155 5155 5155 5155 5155 4004
 5155   92 5155 5155   84   42   45   40   39   45   83   83   83   83
   83   83   83   42   83   51   83   83]
['<ROOT>', 'greece', ',', 'for', 'example', ',', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', '<NULL>', 'turkey', '<NULL>', 'and', '<NULL>', '<NULL>', '<p>:<ROOT>', '<p>:NNP', '<p>:,', '<p>:IN', '<p>:NN', '<p>:,', '<p>:<NULL>', '<p>:<NULL>', '<p>:<NULL>', '<p>:<NULL>', '<p>:<NULL>', '<p>:<NULL>', '<p>:<NULL>', '<p>:NNP', '<p>:<NULL>', '<p>:CC', '<p>:<NULL>', '<p>:<NULL>']


## 3. Model

In [68]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense, Reshape, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

### TODO 4 (Coding):
Build and train a tensroflow keras model to predict an action for each state of of the input. This is a simple classification task. 
- The input and output of the model must match the dimention of x_train and y_train.
- The model must use the provided pretrained embeddings
- The model could comprise of only a feedforward layer and a dropout
- Training loss should be around 0.1 or below, and training categorical_accuracy above 0.94

In [99]:
model = Sequential()
# Write your code here
model.add(Input((36,)))
model.add(Embedding(
    5157,
    50,
    input_length=36,
    embeddings_initializer=tf.keras.initializers.Constant(embeddings),
))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy')

In [109]:
# Write your code here
model.summary()
model.fit(x_train, y_train, batch_size=64, epochs=1)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 36, 50)            257850    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 1024)              1844224   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)             

## 4. Evaluation

For Dependency Parsing, we usually report attachment score of the model for evaluation. There are two possible metrics UAS and LAS.

### TODO 5 (Written and Coding):
Explain how attachment score is calculated and the difference between unlabeled attachment score (UAS) and labeled attachment score (LAS). Which one should we use here?

```
UAS = Percentage of words that get the correct head\
LAS = Percentage of words that get the correct head and label

we use UAS because our model predict only transition which provide only dependency not sematic relation (dependency label)
```

__Answer here__:

Report the score using appropriate metric on dev_set and test_set. The function for calculating scores are provided in parser.parse and the dataset can be passed in as-is.

dev_score:

In [107]:
UAS, dependencies = parser.parse(dev_set, model, minibatch_parse)
UAS

0.7454802965925186

test_score:

In [108]:
UAS, dependencies = parser.parse(test_set, model, minibatch_parse)
UAS

0.7740117746005046

Also, print one sample sentence (in English) in the test set and its predicted dependencies from the model.
You can use __parser.id2tok\[word_id\]__ to lookup the word in English.

__Draw a picture of this sentence with arrows and upload it to my couseville__

In [126]:
print(' '.join([parser.id2tok[id] for id in test_set[7]['word']]))
dependencies[7]

<ROOT> the <UNK> has already <UNK> .


[(2, 1), (5, 4), (5, 3), (5, 2), (5, 6), (0, 5)]